# Podstawy programowania (AD) 2

## Tomasz Rodak


Wykład II

---

# Testy jednostkowe

## Instrukcja `assert`

Instrukcja `assert` jest wykorzystywana do weryfikacji założeń i do testowania kodu. Składnia instrukcji `assert` jest następująca:

```python
assert <warunek>, <opcjonalny komunikat>
```

`<warunek>` jest wyrażeniem logicznym. Jeśli `<warunek>` jest fałszywy, to instrukcja `assert` zgłasza wyjątek `AssertionError`. `<komunikat>` jest łańcuchem znaków, który jest wyświetlany w momencie zgłoszenia wyjątku. 

### Przykład II.1

Piszemy funkcję realizującą klasyczny algorytm szukania maksimum w ciągu liczb. Wykorzystamy instrukcję `assert` do testowania **[niezmiennika pętli](https://en.wikipedia.org/wiki/Loop_invariant)**.

```python
# Path: src/maksimum_loop_inv.py
def maksimum(seq):
    """Funkcja zwraca maksimum z sekwencji."""
    if len(seq) == 0:
        raise ValueError("Sekwencja nie może być pusta.")
    
    maks = seq[0]
    
    for x in seq[1:]:
        if x > maks:
            maks = x
    
    return maks
```

Niezmienik pętli to takie wyrażenie logiczne, które jest prawdziwe:
* przed rozpoczęciem pętli,
* w każdej iteracji,
* po zakończeniu pętli.

Przykładem niezmiennika dla pętli w funkcji `maksimum` jest wyrażenie: *zmienna `maks` jest równa maksimum z dotychczas przetworzonych elementów ciągu*. Niezmiennik ten powinien być cały czas prawdziwy. Fakt ten sprawdzimy za pomocą instrukcji `assert`.

```python
# Path: src/maksimum_loop_inv.py
def maksimum_inv(seq):
    """Funkcja zwraca maksimum z sekwencji."""
    if len(seq) == 0:
        raise ValueError("Sekwencja nie może być pusta.")
    
    maks = seq[0]
    
    # Przed rozpoczęciem pętli maks jest równe pierwszemu elementowi sekwencji.
    assert maks == seq[0], "Niezmienik pętli fałszywy przed rozpoczęciem pętli."
    
    for i, x in enumerate(seq[1:], 1):
        if x > maks:
            maks = x
        # W i-tej iteracji pętli maks jest równe maksimum z seq[:i+1].
        assert maks == max(seq[:i+1]), f"Niezmienik pętli fałszywy w {i}-tej iteracji pętli."
    
    # Po zakończeniu pętli maks jest równe maksimum z całej sekwencji.
    assert maks == max(seq), "Niezmienik pętli fałszywy po zakończeniu pętli."
    
    return maks
```

### Ćwiczenie II.1

W pętli:
    
```python
n = 10
j = n

for i in range(n):
    j -= 1
```

zidentyfikuj niezmienik pętli i zaimplementuj go za pomocą instrukcji `assert`.

## Testy jednostkowe

Testy jednostkowe to testy, które sprawdzają poprawność działania pojedynczych elementów programu, takich jak funkcje, klasy, itp. Wywołuje się je zazwyczaj automatycznie. Jedno z podejść do tworzenia oprogramowania, tzw. [programowanie sterowane testami](https://en.wikipedia.org/wiki/Test-driven_development) (TDD), zakłada, że testy, w tym testy jednostkowe, są pisane przed napisaniem kodu, który mają testować.

### Przykład II.2

Piszemy program do obliczania pola powierzchni trójkąta, gdy dane są długości jego boków. Korzystamy z [twierdzenia Herona](https://pl.wikipedia.org/wiki/Twierdzenie_Herona). Mamy tu dwa zagadnienia:
* napisanie funkcji sprawdzającej, czy dane liczby mogą być długościami boków trójkąta,
* napisanie funkcji obliczającej pole powierzchni trójkąta.

Zadanie zaczynamy od napisania testów jednostkowych dla obu funkcji. Oto propozycja testów:


```python
# Path: src/triangle_area_tests_example/assert_test_triangle_area.py
from triangle_area import is_triangle, triangle_area
from math import sqrt, isclose


def test_is_triangle():
    assert is_triangle(3, 4, 5), "3, 4, 5 jest trójkątem prostokątnym"
    assert is_triangle(1, 1, 1), "Trójkąt równoboczny"
    assert is_triangle(5, 2, 3), "5, 2, 3 jest trójkątem zdegenerowanym"
    assert is_triangle(
        100, 100, sqrt(2) * 100
    ), "100, 100, 100√2 jest trójkątem prostokątnym"
    assert not is_triangle(1, 1, 3), "1, 1, 3 nie jest trójkątem"
    assert not is_triangle(-5, 2, 3), "-5, 2, 3 nie jest trójkątem"
    assert not is_triangle(1, -1, -3), "1, -1, -3 nie jest trójkątem"


def test_triangle_area():
    assert isclose(triangle_area(3, 4, 5), 6), "Pole trójkąta 3, 4, 5"
    assert isclose(triangle_area(1, 1, 1), sqrt(3) / 4), "Pole trójkąta 1, 1, 1"
    assert isclose(triangle_area(5, 2, 3), 0), "Pole trójkąta 5, 2, 3"
    assert isclose(
        triangle_area(100, 100, sqrt(2) * 100), 5000
    ), "Pole trójkąta 100, 100, 100√2"


def test_triangle_area_exceptions():
    try:
        triangle_area(1, 1, 3)
    except ValueError as e:
        assert str(e) == "Nie można zbudować trójkąta o bokach 1, 1, 3"
    else:
        raise AssertionError("Nie zgłoszono wyjątku dla 1, 1, 3")

    try:
        triangle_area(-5, 2, 3)
    except ValueError as e:
        assert str(e) == "Nie można zbudować trójkąta o bokach -5, 2, 3"
    else:
        raise AssertionError("Nie zgłoszono wyjątku dla -5, 2, 3")


def test_all():
    """Uruchamia wszystkie testy"""
    test_is_triangle()
    test_triangle_area()
    test_triangle_area_exceptions()
    print("=== All tests passed. ===")


if __name__ == "__main__":
    test_all()
```

Program `triangle_area.py` zawiera implementację funkcji `is_triangle()` i `triangle_area()`. Funkcja `is_triangle()` sprawdza, czy dane liczby mogą być długościami boków trójkąta i zwraca wartość logiczną. Funkcja `triangle_area()` oblicza pole powierzchni trójkąta o bokach o długościach podanych jako argumenty lub zgłasza wyjątek `ValueError`, jeśli takiego trójkąta zbudować się nie da.

```python
# Path: src/triangle_area_tests_example/triangle_area.py
"""Moduł zawierający funkcje do obliczania pola trójkąta"""

from math import sqrt


def is_triangle(a, b, c):
    """Sprawdza, czy z boków a, b, c można zbudować trójkąt."""
    return a + b >= c and b + c >= a and c + a >= b


def triangle_area(a, b, c):
    """Oblicza pole trójkąta o bokach a, b, c za pomocą wzoru Herona."""
    if not is_triangle(a, b, c):
        raise ValueError(f"Nie można zbudować trójkąta o bokach {a}, {b}, {c}")
    p = (a + b + c) / 2
    return sqrt(p * (p - a) * (p - b) * (p - c))


def main():
    print("Witaj w programie do obliczania pola trójkąta.")
    a = float(input("Podaj długość pierwszego boku: "))
    b = float(input("Podaj długość drugiego boku: "))
    c = float(input("Podaj długość trzeciego boku: "))
    try:
        area = triangle_area(a, b, c)
    except ValueError as e:
        print(e)
    else:
        print(f"Pole trójkąta wynosi {area}")


if __name__ == "__main__":
    main()
```

Programy `triangle_area.py` i `assert_test_triangle_area.py` powinny znajdować się w tym samym katalogu. Test można uruchomić z poziomu edytora, jak każdy inny program, lub z wiersza poleceń za pomocą:

```bash
python assert_test_triangle_area.py
```

Wynik:

```
=== All tests passed. ===
```

### Ćwiczenie II.2

Przeprowadź powyższe postępowanie. Zobacz co się stanie, gdy dokonasz zmian w implementacji funkcji `is_triangle()` lub `triangle_area()` tak, aby testy nie przechodziły.

### Moduł [`unittest`](https://docs.python.org/3/library/unittest.html)

Pisanie i uruchamianie testów tak jak w powyższym przykładzie szybko staje się dość uciążliwe, dlatego powstały narzędzia ułatwiające pisanie i automatyczne uruchamianie testów. Jednym z nich jest moduł `unittest` znajdujący się w bibliotece standardowej Pythona. Poniżej podaję przykład użycia modułu `unittest` do testowania funkcji `is_triangle()` i `triangle_area()`. 

<p style="color: green">
Nie przejmuj się, jeśli poniższy kod jest dla Ciebie nie do końca zrozumiały. W tej chwili chodzi o to, abyś potrafił uruchomić testy jednostkowe za pomocą modułu <code>unittest</code> i wiedział co testuje dany test, czyli potrafił testy czytać. Na dalszych zajęciach omówimy programowanie obiektowe (pisanie klas) i wtedy kody źródłowe testów jednostkowych staną się całkowicie zrozumiałe.
</p>

```python
# Path: src/triangle_area_tests_example/test_triangle_area.py
"""Testy jednostkowe dla modułu triangle_area"""

import unittest
from math import sqrt

from triangle_area import is_triangle, triangle_area


class TestTriangleArea(unittest.TestCase):
    """Testy jednostkowe dla funkcji is_triangle() i triangle_area()."""

    def test_is_triangle(self):
        """Testy pozytywne dla funkcji is_triangle()."""
        self.assertTrue(is_triangle(3, 4, 5), msg="3, 4, 5 jest trójkątem prostokątnym")
        self.assertTrue(is_triangle(1, 1, 1), msg="Trójkąt równoboczny")
        self.assertTrue(
            is_triangle(5, 2, 3), msg="5, 2, 3 jest trójkątem zdegenerowanym"
        )
        self.assertTrue(
            is_triangle(100, 100, sqrt(2) * 100),
            msg="100, 100, 100√2 jest trójkątem prostokątnym",
        )

    def test_is_not_triangle(self):
        """Testy negatywne dla funkcji is_triangle()."""
        self.assertFalse(is_triangle(1, 1, 3), msg="1, 1, 3 nie jest trójkątem")
        self.assertFalse(is_triangle(-5, 2, 3), msg="-5, 2, 3 nie jest trójkątem")
        self.assertFalse(is_triangle(1, -1, -3), msg="1, -1, -3 nie jest trójkątem")

    def test_triangle_area(self):
        """Testy funkcji triangle_area()."""
        self.assertAlmostEqual(triangle_area(3, 4, 5), 6, msg="Pole trójkąta 3, 4, 5")
        self.assertAlmostEqual(
            triangle_area(1, 1, 1), sqrt(3) / 4, msg="Pole trójkąta 1, 1, 1"
        )
        self.assertAlmostEqual(triangle_area(5, 2, 3), 0, msg="Pole trójkąta 5, 2, 3")
        self.assertAlmostEqual(
            triangle_area(100, 100, sqrt(2) * 100),
            5000,
            msg="Pole trójkąta 100, 100, 100√2",
        )

    def test_triangle_area_exceptions(self):
        """Testy wyjątków dla funkcji triangle_area()."""
        with self.assertRaises(ValueError) as e:
            triangle_area(1, 1, 3)
        self.assertEqual(
            str(e.exception), "Nie można zbudować trójkąta o bokach 1, 1, 3"
        )

        with self.assertRaises(ValueError) as e:
            triangle_area(-5, 2, 3)
        self.assertEqual(
            str(e.exception), "Nie można zbudować trójkąta o bokach -5, 2, 3"
        )


if __name__ == "__main__":
    unittest.main()

```

Test uruchamiamy tak samo jak poprzednio, w edytorze lub z wiersza poleceń:

```bash
python test_triangle_area.py
```

Wynik:

```
....
----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK
```

Bardziej wyczerpujące komunikaty o błędach i wynikach testów uzyskamy, jeśli uruchomimy testy z wiersza poleceń z opcją `-v`:

```bash
python -v test_triangle_area.py
```

Wynik:

```
test_is_not_triangle (__main__.TestTriangleArea)
Testy negatywne dla funkcji is_triangle(). ... ok
test_is_triangle (__main__.TestTriangleArea)
Testy pozytywne dla funkcji is_triangle(). ... ok
test_triangle_area (__main__.TestTriangleArea)
Testy funkcji triangle_area(). ... ok
test_triangle_area_exceptions (__main__.TestTriangleArea)
Testy wyjątków dla funkcji triangle_area(). ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.001s

OK
```



## Testy w docstringach. Moduł `doctest`

Moduł `doctest` umożliwia umieszczanie testów jednostkowych w docstringach funkcji, klas i metod. Testy te są fragmentami sesji interaktywnych, które są wykonywane, a wyniki są porównywane z wynikami oczekiwanymi. 

Kod

```python
import doctest

doctest.testmod()
```

testuje wszystkie docstringi w środowisku globalnym.

Domyślnie wykonywane są wszystkie testy, dłuższa informacja wypisywana jest dla tych testów, które się nie powiodły.


### Przykład

Testy w docstringu dla funkcji zwracającej dodatnie dzielniki.

```python
# Path: src/dzielniki_doctest.py
def dzielniki(n):
    '''Zwraca listę dodatnich dzielników n.

    >>> dzielniki(1)
    [1]
    >>> dzielniki(5)
    [1, 5]
    >>> dzielniki(-8)
    [1, 2, 4, 8]
    '''

    pass
```

Mamy gotowy nagłówek funkcji wraz z docstringiem, w którym umieściliśmy testy. Testy te możemy wykonać z wiersza poleceń za pomocą:

```bash
python -m doctest dzielniki_doctest.py
```

Wynik:

```
**********************************************************************
File "/home/tomek/Dokumenty/PPwAD2/src/dzielniki_doctest.py", line 7, in dzielniki_doctest.dzielniki
Failed example:
    dzielniki(1)
Expected:
    [1]
Got nothing
**********************************************************************
File "/home/tomek/Dokumenty/PPwAD2/src/dzielniki_doctest.py", line 9, in dzielniki_doctest.dzielniki
Failed example:
    dzielniki(5)
Expected:
    [1, 5]
Got nothing
**********************************************************************
File "/home/tomek/Dokumenty/PPwAD2/src/dzielniki_doctest.py", line 11, in dzielniki_doctest.dzielniki
Failed example:
    dzielniki(-8)
Expected:
    [1, 2, 4, 8]
Got nothing
**********************************************************************
1 items had failures:
   3 of   3 in dzielniki_doctest.dzielniki
***Test Failed*** 3 failures.
```

Testy nie przeszły, gdyż jeszcze nie napisaliśmy funkcji. Zauważmy w tym miejscu, że przed napisaniem funkcji **żaden** test nie powinien przechodzić. W przeciwnym razie oznaczałoby to, że testy są napisane źle.

Alternatywnie, testy można uruchomić z poziomu edytora, jeśli tylko dopiszemy na końcu pliku:

```python
if __name__ == "__main__":
    import doctest
    doctest.testmod()
```

Plik `dzielniki_doctest.py` po zmianach:

```python
# Path: src/dzielniki_doctest.py
def dzielniki(n):
    """Zwraca listę dodatnich dzielników n.

    Args:
        n (int): liczba całkowita

    >>> dzielniki(1)
    [1]
    >>> dzielniki(5)
    [1, 5]
    >>> dzielniki(-8)
    [1, 2, 4, 8]
    """

    n = n if n > 0 else -n
    return [d for d in range(1, n + 1) if n % d == 0]


if __name__ == "__main__":
    import doctest

    doctest.testmod()
```

Wywołanie `doctest.testmod(verbose=True)` przeprowadza testy w trybie "gadatliwym". Dzięki temu w przypadku sukcesu mamy wizualne potwierdzenie, że testy zostały wykonane. Z wiersza poleceń wywołanie `python -m doctest -v dzielniki_doctest.py` daje podobny efekt.

Na koniec, wywołanie

```python
doctest.run_docstring_examples(f, globals(), verbose=True)
```

wykonuje testy tylko dla obiektu `f`, a nie dla wszystkich docstringów w środowisku globalnym.

# Do poczytania

[EAFP](https://docs.python.org/3/glossary.html#term-eafp) vs [LBYL](https://docs.python.org/3/glossary.html#term-lbyl)

- EAFP -- Easier to ask for forgiveness than permission.
- LBYL -- Look before you leap.
